# SUM RESULTS BY COUNTRY (2018-2019)

-Phân tích và thống kê kết quả thi của thí sinh: Tính toán số môn đã thi và tổng điểm của từng thí sinh để xác định kết quả ("Đậu", "Rớt", "Thi lại").

-Tổng hợp kết quả theo năm và tỉnh: Tổng hợp số lượng thí sinh đậu, rớt, và thi lại theo từng tỉnh cho hai năm 2018 và 2019. Mỗi tỉnh sẽ có thống kê chi tiết về số lượng thí sinh đạt mỗi loại kết quả trong từng năm.

-Kết hợp với dữ liệu tỉnh: Ghép mã tỉnh với tên tỉnh để bảng kết quả dễ hiểu hơn.

-Xuất kết quả ra file CSV: Lưu dữ liệu tổng kết vào file Summary_Result_By_Year.csv để tiện cho việc xem và sử dụng sau này. 

In [ ]:
import pandas as pd
import numpy as np

# Đọc dữ liệu từ Data.csv và Tinh.csv
data_df = pd.read_csv('../Data/Cleaned_Data.csv')
tinh_df = pd.read_csv('../Data/Tinh.csv')

# Tính tổng số môn mỗi thí sinh đã thi và tổng điểm
subjects = ['Toan', 'Van', 'Ly', 'Sinh', 'Ngoai ngu', 'Hoa', 'Lich su', 'Dia ly', 'GDCD']
data_df['Total_Score'] = data_df[subjects].sum(axis=1)
data_df['Subjects_Taken'] = data_df[subjects].notnull().sum(axis=1)

# Xác định kết quả (Đậu/Rớt/Thi lại) cho mỗi thí sinh
conditions = [
    (data_df['Subjects_Taken'] <= 3),  # Thi lại nếu có <= 3 môn
    (data_df['Total_Score'] >= 15)     # Đậu nếu tổng điểm >= 15
]
choices = ["Thi lại", "Đậu"]
data_df['Result'] = np.select(conditions, choices, default="Rớt")

# Lọc dữ liệu theo năm và tính số lượng thí sinh đậu/rớt/thi lại theo tỉnh cho từng năm
summary_2018 = data_df[data_df['Year'] == 2018].groupby(['MaTinh', 'Result']).size().unstack(fill_value=0).reset_index()
summary_2018['Tổng thí sinh 2018'] = summary_2018[['Đậu', 'Rớt', 'Thi lại']].sum(axis=1)

summary_2019 = data_df[data_df['Year'] == 2019].groupby(['MaTinh', 'Result']).size().unstack(fill_value=0).reset_index()
summary_2019['Tổng thí sinh 2019'] = summary_2019[['Đậu', 'Rớt', 'Thi lại']].sum(axis=1)

# Đổi tên cột để phân biệt các năm
summary_2018 = summary_2018.rename(columns={
    'Đậu': 'Số thí sinh đậu 2018',
    'Rớt': 'Số thí sinh rớt 2018',
    'Thi lại': 'Số thí sinh thi lại 2018'
})

summary_2019 = summary_2019.rename(columns={
    'Đậu': 'Số thí sinh đậu 2019',
    'Rớt': 'Số thí sinh rớt 2019',
    'Thi lại': 'Số thí sinh thi lại 2019'
})

# Hợp nhất dữ liệu của hai năm 2018 và 2019
summary_df = pd.merge(summary_2018, summary_2019, on='MaTinh', how='outer').fillna(0)

# Thêm tên tỉnh vào bảng tổng kết
summary_df = pd.merge(summary_df, tinh_df, on='MaTinh', how='left').rename(columns={'TenTinh': 'Tên Tỉnh'})

# Xuất kết quả ra file CSV
summary_file_path = '../output/Summary_Result_By_Year.csv'
summary_df.to_csv(summary_file_path, index=False)

# Hiển thị thông tin
print("Kết quả đã được lưu vào:", summary_file_path)


# SUM RESULTS STUDENTS BY PROVINCE (2018-2019)